In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
from pprint import pprint
import requests

In [2]:
import pandas as pd

df = pd.read_csv('movies_data.csv')

# drop all columns except 'imdb_id'
df = df[['imdb_id']]

# save the newDataFrame to a CSV file
df.to_csv('IMdb_file.csv', index=False)

In [3]:
headers = {
    'Host': 'www.imdb.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'
}

In [4]:
def get_title_details(imdb_id):
    imdb_url = f'https://www.imdb.com/title/{imdb_id.strip()}/'

    page = bs(requests.get(imdb_url, headers=headers).text, 'lxml')
    
    return {
        'Budget': page.find('li', {'data-testid': 'title-boxoffice-budget'}).find('div').text if page.find('li', {'data-testid': 'title-boxoffice-budget'}) else '',
        'Box Office US/CA': page.find('li', {'data-testid': 'title-boxoffice-grossdomestic'}).find('div').text if page.find('li', {'data-testid': 'title-boxoffice-grossdomestic'}) else '',
        'Box Office Worldwide': page.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'}).find('div').text if page.find('li', {'data-testid': 'title-boxoffice-cumulativeworldwidegross'}) else '',
        'Length': page.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'}).find_all('li')[-1].text if page.find('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers sc-afe43def-4 kdXikI baseAlt'}) else '',
        'Rating': page.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'}).find('span').text if page.find('div', {'data-testid': 'hero-rating-bar__aggregate-rating__score'}) else '',
        'Director Names': ', '.join([director.text.strip() for director in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[0].find_all('li')]) if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 1 else '',
        'Director Links': ['https://www.imdb.com' + director.find('a').get('href').split('?')[0] for director in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[0].find_all('li')] if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 1 else '',
        'Cast Names': ', '.join([cast.text.strip() for cast in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[-1].find_all('li')]) if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 0 else '',
        'Cast Links': ['https://www.imdb.com' + director.find('a').get('href').split('?')[0] for director in page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})[-1].find_all('li')] if len(page.find_all('ul', {'class': 'ipc-inline-list ipc-inline-list--show-dividers ipc-inline-list--inline ipc-metadata-list-item__list-content baseAlt'})) > 0 else '',
    }

In [5]:
def get_title_awards(imdb_id):
    imdb_url = f'https://www.imdb.com/title/{imdb_id}/awards/'
    page = bs(requests.get(imdb_url, headers=headers).text, 'lxml')
    
    return ', '.join([award.find_all('td')[-1].text.strip() for award in page.find('table', {'class': 'awards'}).find_all('tr')]) if page.find('table', {'class': 'awards'}) else ''

In [11]:
df = pd.read_csv('IMDb_file.csv')

new_columns_to_add = [
    'Movie Budget',
    'Movie Box Office US/CA',
    'Movie Box Office Worldwide',
    'Movie Length',
    'Movie Rating',
    'Movie Director Names',
    'Movie Director Links',
    'Movie Cast Names',
    'Movie Cast Links',
    'Movie Awards',
   
]

for column in new_columns_to_add:
    if not column in df.columns:
        df.insert(loc=len(df.columns), column=column, value=['']*len(df), allow_duplicates=True)

In [12]:
for x in range(len(df)):
    imdb_id = df.iloc[x]['imdb_id']
    original_details = get_title_details(imdb_id)

    ##rIMDb_id = df.iloc[x]['rIMDb']
    ##remade_details = get_title_details(rIMDb_id)

    for key, value in original_details.items():
        df.at[x, f'Movie {key}'] = value

    #for key, value in remade_details.items():
        #df.at[x, f'Remade {key}'] = value

    df.at[x, 'Movie Awards'] = get_title_awards(imdb_id)
    #df.at[x, 'Remade Awards'] = get_title_awards(rIMDb_id)


    print(df.loc[x])
    df = df.fillna('')

    df.to_csv('SequelOutput.csv', index=None)

imdb_id                                                               tt0795421
Movie Budget                                            $52,000,000 (estimated)
Movie Box Office US/CA                                             $144,169,664
Movie Box Office Worldwide                                         $694,476,782
Movie Length                                                             1h 48m
Movie Rating                                                                6.5
Movie Director Names                                             Phyllida Lloyd
Movie Director Links                     [https://www.imdb.com/name/nm1630273/]
Movie Cast Names                  Meryl Streep, Pierce Brosnan, Amanda Seyfried
Movie Cast Links              [https://www.imdb.com/name/nm0000658/, https:/...
Movie Awards                  Judy Craymer\n\nGary Goetzman\n\nPhyllida Lloy...
Name: 0, dtype: object
imdb_id                                                               tt6911608
Movie Budget     

imdb_id                                                               tt0830515
Movie Budget                                           $200,000,000 (estimated)
Movie Box Office US/CA                                             $168,368,427
Movie Box Office Worldwide                                         $589,580,482
Movie Length                                                             1h 46m
Movie Rating                                                                6.6
Movie Director Names                                               Marc Forster
Movie Director Links                     [https://www.imdb.com/name/nm0286975/]
Movie Cast Names                  Daniel Craig, Olga Kurylenko, Mathieu Amalric
Movie Cast Links              [https://www.imdb.com/name/nm0185819/, https:/...
Movie Awards                  Best Sound\nJimmy Boyle\n\nEddy Joseph\n\nChri...
Name: 10, dtype: object
imdb_id                                                               tt2379713
Movie Budget    

imdb_id                                                               tt0062512
Movie Budget                                             $9,500,000 (estimated)
Movie Box Office US/CA                                              $43,084,787
Movie Box Office Worldwide                                          $43,095,926
Movie Length                                                             1h 57m
Movie Rating                                                                6.8
Movie Director Names                                              Lewis Gilbert
Movie Director Links                     [https://www.imdb.com/name/nm0318150/]
Movie Cast Names                      Sean Connery, Akiko Wakabayashi, Mie Hama
Movie Cast Links              [https://www.imdb.com/name/nm0000125/, https:/...
Movie Awards                      Best British Art Direction (Colour)\nKen Adam
Name: 20, dtype: object
imdb_id                                                               tt0064757
Movie Budget    

imdb_id                                                               tt1099212
Movie Budget                                            $37,000,000 (estimated)
Movie Box Office US/CA                                             $193,962,473
Movie Box Office Worldwide                                         $408,430,415
Movie Length                                                              2h 2m
Movie Rating                                                                5.3
Movie Director Names                                        Catherine Hardwicke
Movie Director Links                     [https://www.imdb.com/name/nm0362566/]
Movie Cast Names                 Kristen Stewart, Robert Pattinson, Billy Burke
Movie Cast Links              [https://www.imdb.com/name/nm0829576/, https:/...
Movie Awards                                                  Best Fantasy Film
Name: 30, dtype: object
imdb_id                                                               tt1259571
Movie Budget    

imdb_id                                                               tt0359013
Movie Budget                                            $65,000,000 (estimated)
Movie Box Office US/CA                                              $52,411,906
Movie Box Office Worldwide                                         $131,977,904
Movie Length                                                             1h 53m
Movie Rating                                                                5.8
Movie Director Names                                             David S. Goyer
Movie Director Links                     [https://www.imdb.com/name/nm0275286/]
Movie Cast Names                Wesley Snipes, Kris Kristofferson, Parker Posey
Movie Cast Links              [https://www.imdb.com/name/nm0000648/, https:/...
Movie Awards                                                   Best Horror Film
Name: 40, dtype: object
imdb_id                                                               tt0105698
Movie Budget    

...

imdb_id                                                               tt3477064
Movie Budget                                                                   
Movie Box Office US/CA                                                         
Movie Box Office Worldwide                                          $39,477,906
Movie Length                                                             1h 57m
Movie Rating                                                                3.8
Movie Director Names                             Togan Gökbakar, Emre Karademir
Movie Director Links          [https://www.imdb.com/name/nm2204103/, https:/...
Movie Cast Names                  Sahan Gökbakar, Firuze Gamze Aksu, Gamze Aksu
Movie Cast Links              [https://www.imdb.com/name/nm2201881/, https:/...
Movie Awards                                                                   
Name: 3400, dtype: object
imdb_id                                                               tt1577061
Movie Budget  

imdb_id                                                               tt5813916
Movie Budget                                          TRL 2,000,000 (estimated)
Movie Box Office US/CA                                                         
Movie Box Office Worldwide                                          $11,378,898
Movie Length                                                             2h 15m
Movie Rating                                                                8.2
Movie Director Names                                               Alper Caglar
Movie Director Links                     [https://www.imdb.com/name/nm3809021/]
Movie Cast Names                 Caglar Ertugrul, Ufuk Bayraktar, Ahu Türkpençe
Movie Cast Links              [https://www.imdb.com/name/nm5384553/, https:/...
Movie Awards                      Best Movie, Best Movie Director\nAlper Caglar
Name: 3410, dtype: object
imdb_id                                                               tt0493264
Movie Budget  

imdb_id                                                               tt2645104
Movie Budget                                             $2,500,000 (estimated)
Movie Box Office US/CA                                                         
Movie Box Office Worldwide                                           $8,701,745
Movie Length                                                             1h 45m
Movie Rating                                                                5.1
Movie Director Names                                                Erol Özlevi
Movie Director Links                     [https://www.imdb.com/name/nm5482023/]
Movie Cast Names                  Sinem Kobal, Engin Altan Düzyatan, Sedef Avci
Movie Cast Links              [https://www.imdb.com/name/nm1109740/, https:/...
Movie Awards                                                                   
Name: 3420, dtype: object
imdb_id                                                               tt1092011
Movie Budget  

imdb_id                                                               tt0274931
Movie Budget                                                                   
Movie Box Office US/CA                                                         
Movie Box Office Worldwide                                                     
Movie Length                                                             1h 21m
Movie Rating                                                                6.6
Movie Director Names                                               Tunç Basaran
Movie Director Links                     [https://www.imdb.com/name/nm0059633/]
Movie Cast Names                         Kartal Tibet, Zuhal Aktan, Lale Belkis
Movie Cast Links              [https://www.imdb.com/name/nm0862605/, https:/...
Movie Awards                                                                   
Name: 3430, dtype: object
imdb_id                                                               tt0274932
Movie Budget  

imdb_id                                                               tt1630038
Movie Budget                                             $1,500,000 (estimated)
Movie Box Office US/CA                                                 $165,987
Movie Box Office Worldwide                                             $165,987
Movie Length                                                             1h 35m
Movie Rating                                                                6.6
Movie Director Names                                             Charlie Nguyen
Movie Director Links                     [https://www.imdb.com/name/nm1110306/]
Movie Cast Names                            Dustin Nguyen, Kathy Uyen, Thai Hoa
Movie Cast Links              [https://www.imdb.com/name/nm0629006/, https:/...
Movie Awards                                                                   
Name: 3440, dtype: object
imdb_id                                                               tt3701078
Movie Budget  